In [13]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

data_dir = Path("..", "data")

In [1]:


# ok, we have a folder of .parquet files - with a ngram, and a count column
# we want to end with one big file, with a ngram, and a count column
# but there is duplicaiton between the fields, and they are lots of them

# We can't load them all at once, so we need open up the first, and then sequentially update it
# We can't just append, because we need to update the counts - so it'll need a groupby and aggregate the count

paths = list(data_dir.glob("*.parquet"))

# open the first file
df = pd.read_parquet(paths[0])
# drop any with a count of one
# shortcut! but for our purposes ok
df = df[df["count"] > 1]

# loop through the rest

for item_path in tqdm(paths[1:]):
    item = pd.read_parquet(item_path)
    item = item[item["count"] > 1]
    df = pd.concat([df, item])
    df = df.groupby("ngram").sum().reset_index()

df.head()

100%|██████████| 38/38 [10:08<00:00, 16.00s/it]


,ngram,count
0,! !,21
1,! ! !,10
2,! ! ! !,4
3,! ! ! ”,4
4,! ! ”,4


In [15]:
df = pd.read_parquet(data_dir / "joined_ngram.parquet")

df = df[df["count"] > 10]

df.to_parquet(data_dir / "joined_ngram_reduced.parquet")